<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

/var/folders/b5/g9qh0pv107q2zdxqjdmjv97m0000gn/T/ipykernel_4969/1481237235.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [11]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [12]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence


sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
# for _, row in df[:None].iterrows():
#     sentence_tokens.append(text_to_word_sequence(row[0]))
sentence_tokens = df[0].apply(text_to_word_sequence).tolist()


In [13]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [51]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [15]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [16]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [17]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [18]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [19]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.1484375
Loss after epoch 1: 65953.9609375
Loss after epoch 2: 65934.0
Loss after epoch 3: 65712.109375
Loss after epoch 4: 63872.4375
Loss after epoch 5: 64142.15625
Loss after epoch 6: 64058.90625
Loss after epoch 7: 64755.53125
Loss after epoch 8: 62585.6875
Loss after epoch 9: 60415.1875
Loss after epoch 10: 59823.6875
Loss after epoch 11: 58929.875
Loss after epoch 12: 57726.125
Loss after epoch 13: 56493.0
Loss after epoch 14: 55842.3125
Loss after epoch 15: 55862.6875
Loss after epoch 16: 51696.0625
Loss after epoch 17: 49815.75
Loss after epoch 18: 49568.125
Loss after epoch 19: 48959.75


(156986, 287740)

### 4 - Ensayar

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087111949920654),
 ('sleep', 0.8649654388427734),
 ('help', 0.859427273273468),
 ('try', 0.8455997705459595),
 ('cry', 0.8387208580970764),
 ('little', 0.827362060546875),
 ('not', 0.8181124329566956),
 ('seems', 0.8175204992294312),
 ('twist', 0.8154967427253723),
 ('peace', 0.8122625946998596)]

In [21]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.2178674340248108),
 ('our', -0.24496109783649445),
 ('come', -0.26132795214653015),
 ('on', -0.26722264289855957),
 ('bang', -0.2733001112937927),
 ('five', -0.27418217062950134),
 ('six', -0.27740582823753357),
 ('baby', -0.2776320278644562),
 ('work', -0.27903637290000916),
 ('four', -0.2792740762233734)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9786691069602966),
 ('three', 0.9761053919792175),
 ('six', 0.9668909907341003),
 ('two', 0.9565711617469788),
 ('seven', 0.954231858253479),
 ('sixty', 0.9003376960754395),
 ('one', 0.8165019750595093),
 ('us', 0.7762883305549622),
 ('crying', 0.7727385759353638),
 ('strawberry', 0.7672423720359802)]

In [23]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409590363502502),
 ('buy', 0.9399516582489014),
 ('much', 0.8996817469596863),
 ('hide', 0.8465147018432617),
 ('just', 0.8415088057518005)]

In [24]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [68]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736575  0.2146101  -0.05407884  0.06707156  0.13060287 -0.2571361
  0.00711569  0.26743898 -0.03100282  0.259966    0.21179484 -0.08465589
 -0.26272163  0.30837908 -0.07934508 -0.13357273  0.29947317  0.18061721
 -0.13111906 -0.11857361  0.03467939  0.11256406  0.08478606  0.08090245
 -0.1762275   0.03122391 -0.08726044  0.23747045 -0.11507294 -0.29118884
 -0.03650349 -0.33222473  0.2326327   0.10366983 -0.18365678  0.08299293
  0.14315957 -0.19761501 -0.00757889  0.11195931 -0.10420218 -0.13083498
 -0.03264751 -0.04687788  0.11331188  0.10231608 -0.16201344 -0.10347553
  0.03660087 -0.08524201 -0.34872565 -0.02154043  0.25938827  0.16540739
  0.03108748  0.09505356  0.24453093 -0.09860147  0.18298486  0.06455195
  0.01696305 -0.3259232  -0.04773364 -0.05418696 -0.04338877 -0.09137462
  0.00889946  0.1163592  -0.25662434  0.05426565  0.08916582  0.04541196
  0.20835699 -0.19101447  0.32209986  0.19337751  0.09080258 -0.00362502
 -0.18663995 -0.07053484 -0.12137373 -0.0347695  -0.

In [69]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 0.9999999403953552),
 ('babe', 0.9132977724075317),
 ('someone', 0.8914989829063416),
 ('nothing', 0.8811892867088318),
 ('need', 0.8758766055107117),
 ("didn't", 0.8689667582511902),
 ("there's", 0.8557892441749573),
 ('you', 0.8447702527046204),
 ('feed', 0.8422104120254517),
 ('somebody', 0.8385516405105591)]

In [70]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9132977724075317),
 ('someone', 0.8914989233016968),
 ('nothing', 0.8811892867088318),
 ('need', 0.8758766055107117),
 ("didn't", 0.8689666986465454),
 ("there's", 0.8557891845703125),
 ('you', 0.8447702527046204),
 ('feed', 0.8422104120254517),
 ('somebody', 0.8385516405105591),
 ('hope', 0.8361444473266602)]

### 5 - Visualizar agrupación de vectores

In [71]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [72]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [73]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [74]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

## Resolución del desafío

### Ejercicio 
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.


#### Carga del dataset

In [31]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec

In [32]:
# Elijo como dataset las novelas de Harry Potter disponibles Hugging Face


# Load dataset from HF
dataset = load_dataset("elricwan/HarryPotter", split="train")

# Convert to pandas DataFrame if needed
df_HarryPotter = dataset.to_pandas()

print(df_HarryPotter.head())

                                          filename  \
0      1-Harry-Potter-and-the-Sorcerer’s-Stone.txt   
1    2-Harry-Potter-and-the-Chamber-of-Secrets.txt   
2   3-Harry-Potter-and-the-Prisoner-of-Azkaban.txt   
3        4-Harry-Potter-and-the-Goblet-of-Fire.txt   
4  5-Harry-Potter-and-the-Order-of-the-Phoenix.txt   

                                             content  
0  FOR JESSICA, WHO LOVES STORIES,\n\nFOR ANNE, W...  
1  FOR SEÁN P. F. HARRIS,\n\nGETAWAY DRIVER AND F...  
2  TO JILL PREWETT AND\n\nAINE KIELY,\n\nTHE GODM...  
3  TO PETER ROWLING,\n\nIN MEMORY OF MR. RIDLEY\n...  
4  TO NEIL, JESSICA, AND DAVID,\n\nWHO MAKE MY WO...  


In [33]:
print("Cantidad de documentos:", df_HarryPotter.shape[0])
print("Nombre de las columnas del DF: ", df_HarryPotter.columns)


Cantidad de documentos: 8
Nombre de las columnas del DF:  Index(['filename', 'content'], dtype='object')


## Observaciones:
El dataset presenta 8 archivos de texto correspondientes a los 8 libros de Harry Potter.
Debido a las diferencias con el dataset visto en clase, se utilizarán las siguientes definiciones:
 1) Se concatenarán todos los libros para formar el corpus.
 2) Se considerarán a los párrafos como los documentos para realizar los embeddings.

In [ ]:
# Verificar los documentos, y si se puede dividirlos en párrafos
sentence_tokens_hp = []

for book in df_HarryPotter['content']:
    paragraphs = book.split("\n")  # Dividir el texto en párrafos usando el salto de línea como separador
    for paragraph in paragraphs:
        tokens = preprocess_text(paragraph)
        if tokens:  # Solo agregar si no está vacío
            sentence_tokens_hp.append(tokens)

print(f"Total 'documents': {len(sentence_tokens_hp)}")
print("First doc:", sentence_tokens_hp[0][:50])


Total 'documents': 80066
First doc: ['jessica', 'loves', 'stories']


In [43]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """
    Remueve puntuación, convierte a minúsculas, elimina stopwords y tokeniza el texto.
    """
    # Lowercase
    text = text.lower()
    
    # Keep only alphabetic characters and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Split into words
    tokens = text.split()
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    return tokens


In [44]:
# Aplicar el preprocesamiento a cada parragrafo
# 1) Concatenar todo el texto
all_text = " ".join(df_HarryPotter['content'].tolist())

# Separar por párrafos
paragraphs = all_text.split("\n\n")

# Preprocesar cada párrafo
sentence_tokens_hp = [preprocess_text(p) for p in paragraphs if p.strip()]

print("Ejemplo de párrafo tokenizado:", sentence_tokens_hp[0][:30])
print("Total de párrafos procesados:", len(sentence_tokens_hp))


Ejemplo de párrafo tokenizado: ['jessica', 'loves', 'stories']
Total de párrafos procesados: 80570


In [45]:
from gensim.models import Word2Vec

model_hp_potter = Word2Vec(
    sentences=sentence_tokens_hp,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=1
)


In [48]:
# Obtener el vocabulario con los tokens
model_hp_potter.build_vocab(sentence_tokens_hp)

In [49]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", model_hp_potter.corpus_count)

Cantidad de docs en el corpus: 80570


In [50]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(model_hp_potter.wv.index_to_key))

Cantidad de words distintas en el corpus: 12352


In [52]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
model_hp_potter.train(sentence_tokens_hp,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 3503233.0
Loss after epoch 1: 2937823.5
Loss after epoch 2: 2681756.5
Loss after epoch 3: 2371201.0
Loss after epoch 4: 2366963.0
Loss after epoch 5: 2357709.0
Loss after epoch 6: 2147794.0
Loss after epoch 7: 2027990.0
Loss after epoch 8: 2020886.0
Loss after epoch 9: 2001460.0
Loss after epoch 10: 2002642.0
Loss after epoch 11: 1986294.0
Loss after epoch 12: 1981168.0
Loss after epoch 13: 1982456.0
Loss after epoch 14: 1628368.0
Loss after epoch 15: 1085440.0
Loss after epoch 16: 1100964.0
Loss after epoch 17: 1090916.0
Loss after epoch 18: 1089840.0
Loss after epoch 19: 1064960.0
Loss after epoch 20: 893336.0
Loss after epoch 21: 1068716.0
Loss after epoch 22: 1072112.0
Loss after epoch 23: 1056140.0
Loss after epoch 24: 1059964.0
Loss after epoch 25: 1050528.0
Loss after epoch 26: 1051356.0
Loss after epoch 27: 1052940.0
Loss after epoch 28: 1066980.0
Loss after epoch 29: 1068544.0
Loss after epoch 30: 1085396.0
Loss after epoch 31: 1089020.0
Loss after epoch 32

(55162911, 60602400)

### Procedemos a ensayar y a buscar relaciones de palabras en los documentos

In [53]:
# Palabras que MÁS se relacionan con...:
model_hp_potter.wv.most_similar(positive=["harry"], topn=10)

[('ron', 0.8027319312095642),
 ('hermione', 0.7725063562393188),
 ('shhhh', 0.673701286315918),
 ('cruc', 0.6548905968666077),
 ('tuney', 0.6505980491638184),
 ('jubilantly', 0.6279476881027222),
 ('back', 0.6265587210655212),
 ('noncommittal', 0.6192820072174072),
 ('said', 0.6111220121383667),
 ('throttle', 0.6067759990692139)]

In [55]:
# Palabras que MENOS se relacionan con...:
model_hp_potter.wv.most_similar(negative=["magic"], topn=10)

[('katie', 0.16288213431835175),
 ('shrieked', 0.11411089450120926),
 ('cutting', 0.11048087477684021),
 ('gestured', 0.10700314491987228),
 ('filled', 0.10297875851392746),
 ('smiles', 0.09358055144548416),
 ('trailed', 0.09137681126594543),
 ('noisily', 0.0849495455622673),
 ('gulping', 0.0808401107788086),
 ('accidentally', 0.07704715430736542)]

In [56]:
# Palabras que MÁS se relacionan con...:
model_hp_potter.wv.most_similar(positive=["voldemort"], topn=10)

[('lord', 0.7959990501403809),
 ('voldemorts', 0.7609379291534424),
 ('rookwoods', 0.5961538553237915),
 ('sanctuary', 0.5835540294647217),
 ('feared', 0.5763315558433533),
 ('understands', 0.5762743949890137),
 ('vanquish', 0.5758090615272522),
 ('hoodwinked', 0.5681061148643494),
 ('downfall', 0.5618146657943726),
 ('conquered', 0.5535776615142822)]

In [57]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["flor"])

KeyError: "Key 'flor' not present in vocabulary"

In [61]:
# Palabras que MÁS se relacionan con...:
model_hp_potter.wv.most_similar(positive=["station"], topn=10)

[('cross', 0.6059896349906921),
 ('barrier', 0.5752648115158081),
 ('kings', 0.5721648931503296),
 ('platform', 0.5527564883232117),
 ('threequarters', 0.5314339995384216),
 ('jostled', 0.5269558429718018),
 ('road', 0.5267975926399231),
 ('charing', 0.5253555774688721),
 ('trolleys', 0.5102257132530212),
 ('garage', 0.5060897469520569)]

In [62]:
# el método `get_vector` permite obtener los vectores:
vector_potter = model_hp_potter.wv.get_vector("potter")
print(vector_potter)

[ 5.50663412e-01 -1.52389482e-01 -2.83300012e-01 -1.30949676e-01
  2.27266535e-01 -3.01922768e-01  2.74379283e-01  7.44201303e-01
 -1.17931515e-01 -2.36827713e-02 -5.01735091e-01  6.31573349e-02
  2.43052617e-01  4.56351519e-01  4.09058452e-01 -3.01888939e-02
  1.53085575e-01  1.48005545e-01 -5.28157830e-01 -1.33686781e-01
 -1.32122695e-01  7.00680494e-01 -2.92728422e-04 -4.66681510e-01
 -6.85469881e-02 -3.95182103e-01  2.48562083e-01 -8.27288449e-01
 -4.32975233e-01  1.09445110e-01  5.91463149e-01 -7.13975728e-03
  1.46514401e-01 -4.52105403e-01  1.83359265e-01  3.31424057e-01
 -2.69755155e-01 -2.91276127e-01 -4.82715160e-01 -7.07179368e-01
  3.65084410e-02 -3.25013041e-01  4.01991397e-01  2.10347306e-02
  8.13915357e-02 -1.65663213e-01  9.40462425e-02 -2.03326911e-01
  4.37827379e-01  5.93954504e-01 -5.07086098e-01 -1.13183837e-02
 -3.08690041e-01  1.19622327e-01  1.82093233e-01  2.16655135e-02
 -2.44084865e-01 -4.95229393e-01 -5.94993532e-02 -1.68379575e-01
  3.15947533e-02  6.68947

### Visualización de los vectores:

In [63]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [76]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(model_hp_potter)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() 

In [77]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(model_hp_potter,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() 

## Conclusiones:
